In [1]:
from transformers import PreTrainedTokenizerFast, GenerationConfig , PhiForCausalLM, pipeline
import torch

# 模型保存路径
这里加载sft后的模型和dpo后的模型，看看两者输出的区别

In [2]:
tokenizer_dir = './model_save/tokenizer/'
dpo_model_save_dir = './model_save/dpo/'
sft_model_save_dir = './model_save/sft/'
max_seq_len = 320

## 加载模型

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

In [ ]:
tokenizer = PreTrainedTokenizerFast.from_pretrained(tokenizer_dir)
dpo_model = PhiForCausalLM.from_pretrained(dpo_model_save_dir).to(device)

sft_mdodel = PhiForCausalLM.from_pretrained(sft_model_save_dir).to(device)

model_size = sum(t.numel() for t in dpo_model.parameters())
print(f"Phi-2 size: {model_size / 1000**2:.1f}M parameters")

## 定义两个text-generation的pipeline

In [5]:
dpo_pipe = pipeline(
    "text-generation", 
    model=dpo_model, 
    device=device,
    tokenizer=tokenizer,
)

sft_pipe = pipeline(
    "text-generation", 
    model=sft_mdodel, 
    device=device,
    tokenizer=tokenizer,
)

## 模型输出

In [ ]:
txt = '感冒了要怎么办？'
prompt = f"##提问:\n{txt}\n##回答:\n"
sft_outputs = sft_pipe(prompt, num_return_sequences=1, max_new_tokens=256, pad_token_id=tokenizer.eos_token_id)
dpo_outputs = dpo_pipe(prompt, num_return_sequences=1, max_new_tokens=256, pad_token_id=tokenizer.eos_token_id)


print(f"model after sft output:\n\n{sft_outputs[0]['generated_text']}")
print('\n==================\n')
print(f"model after dpo output:\n\n{dpo_outputs[0]['generated_text']}")

## 不用pipeline组件，使用greedy search方法手动生成

In [7]:
# greedy search
gen_conf = GenerationConfig(
    num_beams=1,
    do_sample=False,
    max_length=320,
    max_new_tokens=256,
    no_repeat_ngram_size=4,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
)

# top k
top_k_gen_conf = GenerationConfig(
    do_sample=True,
    top_k=100,
    max_length=320,
    max_new_tokens=256,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
)

# top p
top_p_gen_conf = GenerationConfig(
    do_sample=True,
    top_k=0,
    top_p=0.95,
    max_length=320,
    max_new_tokens=256,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
)

In [ ]:
tokend = tokenizer.encode_plus(text=prompt)
input_ids, attention_mask = torch.LongTensor([tokend.input_ids]).to(device), torch.LongTensor([tokend.attention_mask]).to(device)

outputs = dpo_model.generate(
    inputs=input_ids,
    attention_mask=attention_mask,
    generation_config=gen_conf,
)

outs = tokenizer.decode(outputs[0].cpu().numpy(), clean_up_tokenization_spaces=True, skip_special_tokens=True,)
print(outs)

# 计算困惑度（perplexity）

In [4]:
from evaluate import load
perplexity = load("perplexity", module_type="metric")
dpo_model_save_dir = './model_save/dpo/'

In [ ]:
results = perplexity.compute(predictions=['生成一个关于食品的描述，包括其因素、口感和用途。', '在感冒期间，最好保持充足的水分和休息，以缓解身体的疲劳。'], add_start_token=False, model_id=dpo_model_save_dir)

In [ ]:
print(results)